In [7]:
import cv2
import numpy as np

def remove_watermark(image_path, output_path="watermark_removed.jpg"):
    """
    Remove semi-transparent watermarks from images with improved accuracy.
    """
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return
    
    h, w = img.shape[:2]
    
    # Create mask using multiple detection methods
    mask = np.zeros((h, w), dtype=np.uint8)
    
    # Method 1: Detect bright/white regions (for light watermarks)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bright_mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    
    # Method 2: Detect low saturation regions (watermarks are often desaturated)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    _, sat_mask = cv2.threshold(hsv[:,:,1], 30, 255, cv2.THRESH_BINARY_INV)
    
    # Method 3: Edge detection to find watermark boundaries
    edges = cv2.Canny(gray, 50, 150)
    edges_dilated = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=1)
    
    # Combine methods: regions that are bright AND low saturation
    combined_mask = cv2.bitwise_and(bright_mask, sat_mask)
    
    # Focus more on corner regions where watermarks typically appear
    corner_weight = np.zeros((h, w), dtype=np.float32)
    corner_size = 0.4  # Check 40% of image from each corner
    
    # Bottom-right corner (most common)
    corner_weight[int(h*(1-corner_size)):, int(w*(1-corner_size)):] = 2.0
    # Top-right corner
    corner_weight[:int(h*corner_size), int(w*(1-corner_size)):] = 1.5
    # Bottom-left corner
    corner_weight[int(h*(1-corner_size)):, :int(w*corner_size)] = 1.5
    # Center-right
    corner_weight[int(h*0.3):int(h*0.7), int(w*0.6):] = 1.3
    
    # Apply corner weighting
    corner_weight = (corner_weight * 127).astype(np.uint8)
    combined_mask = cv2.addWeighted(combined_mask, 0.7, corner_weight, 0.3, 0)
    _, combined_mask = cv2.threshold(combined_mask, 100, 255, cv2.THRESH_BINARY)
    
    # Morphological operations to refine mask
    kernel_small = np.ones((3, 3), np.uint8)
    kernel_large = np.ones((7, 7), np.uint8)
    
    # Remove noise
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel_small)
    # Connect nearby regions
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel_large)
    # Expand mask slightly to ensure full coverage
    combined_mask = cv2.dilate(combined_mask, kernel_small, iterations=2)
    
    # Optional: Remove small isolated regions (likely noise, not watermark)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100:  # Remove very small regions
            cv2.drawContours(combined_mask, [contour], -1, 0, -1)
    
    # Inpaint using the mask
    result = cv2.inpaint(img, combined_mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
    
    # Save results
    cv2.imwrite(output_path, result)
    cv2.imwrite("mask_debug.jpg", combined_mask)  # Save mask for debugging
    
    print(f"✓ Watermark removed successfully!")
    print(f"✓ Result saved as: {output_path}")
    print(f"✓ Debug mask saved as: mask_debug.jpg")
    
    return result, combined_mask


# Usage
if __name__ == "__main__":
    # Replace with your image path
    image_path = "/home/web-h-063/Documents/dishto/notebooks/unnamed (3).jpg"
    
    result, mask = remove_watermark(image_path)
    
    # Optional: Display results if running in an environment with display
    try:
        cv2.imshow("Original", cv2.imread(image_path))
        cv2.imshow("Mask", mask)
        cv2.imshow("Result", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except:
        print("Display not available (running in headless mode)")

✓ Watermark removed successfully!
✓ Result saved as: watermark_removed.jpg
✓ Debug mask saved as: mask_debug.jpg
Display not available (running in headless mode)
